## Lectura de los datos

#### Librerias

In [1]:
import pandas as pd
import numpy as np
import keras_metrics
import tensorflow as tf
import shutil, os, sys
import matplotlib.pyplot as plt
import cv2


from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers, layers
from tensorflow.keras import utils as k
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory,image




#### Datos

In [2]:

file= 'metadata.csv'
dt=pd.read_csv(file, sep=",")

dt.head(1)

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,in_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 28
0,2,0.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN


### Preprocesamiento

Vamos a ver en que proporcion encontramos las muestras de covid y las de otras enfermedades para que la red no se sobre entrene con un solo tipo de imagenes.

In [3]:
covid=[]
otro=[]

for i,r in dt.iterrows():
    if r['finding'] == 'COVID-19':
        covid.append(r)
    else:
        otro.append(r)

print("Co: ",len(covid) )
print("Otro: ",len(otro) )
print("Proporcion de Covid: ", len(covid)/(len(covid)+len(otro))*100)
    


Co:  296
Otro:  76
Proporcion de Covid:  79.56989247311827


Para que la funcion de keras: <i>image_dataset_from_directory</i> funcione para poder entrenar como queremos, separamos las fotos en dos capetas, una para covid y otra para el resto de enfermedades.

Como hemos visto hay muchas más radiografias de covid que de las otras enfermedades así que vamos a igualar el numero de muestras para cada clase.

##### Creamos los directorios

In [4]:
carpeta = os.getcwd()+os.sep+'images'

if not os.path.isdir(carpeta+os.sep+'covid') :
    os.mkdir(carpeta+os.sep+'covid')
if not os.path.isdir(carpeta+os.sep+'covid'+os.sep+'covid'):
    os.mkdir(carpeta+os.sep+'covid'+os.sep+'covid')
    
if not os.path.isdir(carpeta+os.sep+'otro') :
    os.mkdir(carpeta+os.sep+'otro')
if not os.path.isdir(carpeta+os.sep+'otro'+os.sep+'otro'):
    os.mkdir(carpeta+os.sep+'otro'+os.sep+'otro')
        
    

if not os.path.isdir(os.getcwd()+os.sep+'perdidos') :
    os.mkdir(os.getcwd()+os.sep+'perdidos')
    if not os.path.isdir(os.getcwd()+os.sep+'perdidos'+os.sep+'covid'):
        os.mkdir(os.getcwd()+os.sep+'perdidos'+os.sep+'covid')
        if not os.path.isdir(os.getcwd()+os.sep+'perdidos'+os.sep+'otro'):
            os.mkdir(os.getcwd()+os.sep+'perdidos'+os.sep+'otro')
            
            
if not os.path.isdir(os.getcwd()+os.sep+'nuevas') :
    os.mkdir(os.getcwd()+os.sep+'nuevas')
    if not os.path.isdir(os.getcwd()+os.sep+'nuevas'+os.sep+'covid'):
        os.mkdir(os.getcwd()+os.sep+'nuevas'+os.sep+'covid')
        if not os.path.isdir(os.getcwd()+os.sep+'nuevas'+os.sep+'otro'):
            os.mkdir(os.getcwd()+os.sep+'nuevas'+os.sep+'otro')
            

In [5]:
dimensiones=[1920,1500]


##### Movemos las imagenes

In [6]:

foto=''
destino=''
ruta=carpeta+destino
noEstanco=[]
noEstanot=[]
NombresEntrenoCo=[]
NombresEntrenoOt=[]

numcovid=0  #flag para equilibrar las muestras
numvalco=0 #flag para coger muestras diferentes en prediccion covid
numvalotro=0 #flag para coger muestras diferentes en prediccion otra enfermedad
flag=0 #flag para no mover a las carpetas de images y que no aparezca en los dos directorios
pacienteAnterior=0

fotosCo=0
fotosOt=0

for i,r in dt.iterrows():
    flag=0
    foto=r['filename']
    if r['finding'] == 'COVID-19': #si es covid
        if numcovid<(len(otro)+10):
            destino='covid'
            numcovid+=1
            
        elif numvalco < 6 and r['patientid'] != pacienteAnterior: #ejemplos de covid para prediccion
            if os.path.exists(ruta+os.sep+foto):
                shutil.move(ruta+os.sep+foto,os.getcwd()+os.sep+'perdidos'+os.sep+'covid')
                numvalco+=1
                flag=1
                noEstanco.append(foto)
                
    else: #Otra enfermedad
        if numvalotro < 6 and r['patientid'] != pacienteAnterior:#ejemplos de  otra enfermedad para prediccion
            if os.path.exists(ruta+os.sep+foto):
                shutil.move(ruta+os.sep+foto,os.getcwd()+os.sep+'perdidos'+os.sep+'otro')
                numvalotro+=1
                flag=1
                noEstanot.append(foto)
        destino='otro'
        
    if os.path.exists(ruta+os.sep+foto) and flag!=1:
        shutil.move(ruta+os.sep+foto,ruta+os.sep+destino+os.sep+destino)#+os.sep+destino
        #para aumentar los datos
        if destino=='covid':
            fotosCo+=1
        else:
            fotosOt+=1
    pacienteAnterior=r['patientid']
    #print(ruta+os.sep+foto)
    

##### Aumentamos la caantidad de muestras que tenemos

In [7]:

aumento = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(dimensiones[0],dimensiones[1],3)),
    layers.experimental.preprocessing.RandomRotation(0.15),
    layers.experimental.preprocessing.RandomZoom(0.15),
  ]
)


nuevo=os.getcwd()+os.sep+'nuevas'
df=image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,shear_range=0.15, zoom_range=0.1,channel_shift_range = 10, horizontal_flip=True)
#resul=image.DirectoryIterator(directory=carpeta, image_data_generator=df,target_size=(dimensiones[0],dimensiones[1]), color_mode='rgb', subset='training'
#                        class_mode='binary',batch_size=5)
#print(len(resul))
#imagen=df.flow_from_directory(carpeta, target_size=(dimensiones[0],dimensiones[1]), save_to_dir=nuevo, save_prefix='Nuevas', classes=['covid', 'otro'],class_mode="binary",batch_size=1, seed=1234)
#save_prefix='Nuevas',save_format='jpeg',



cov=image.DirectoryIterator(directory=carpeta+os.sep+'covid',image_data_generator=df,target_size=(dimensiones[0],dimensiones[1]), color_mode='rgb',
                        class_mode=None,save_to_dir=nuevo+os.sep+'covid', save_prefix='Nuevas', batch_size=5,save_format='jpeg')
otr=image.DirectoryIterator(directory=carpeta+os.sep+'otro',image_data_generator=df,target_size=(dimensiones[0],dimensiones[1]), color_mode='rgb',
                        class_mode=None,save_to_dir=nuevo+os.sep+'otro', save_prefix='Nuevas', batch_size=5,save_format='jpeg')



#imagen.next()
#entreno=df.fit(imagen, rounds=4)


#l=[]
#for j in range(len(NombresEntrenoCo)):
#    for i in range(9):
#        #carpeta+os.sep+'covid'+os.sep+NombresEntrenoCo[j]
#        imagen=image.load_img(carpeta+os.sep+'covid'+os.sep+NombresEntrenoCo[j], grayscale=False, color_mode='rgb', target_size=(dimensiones[0],dimensiones[1]))
#        imagen=image.img_to_array(imagen)
#        imagen=np.array([imagen])
#        df.flow(imagen,batch_size=5, save_to_dir=os.getcwd()+os.sep+"jk", save_format='jpg',shuffle=True,save_prefix='Nuevas')
        #nueva = aumento(imagen)
        #cv2.imwrite(carpeta+os.sep+"covid",nueva[0].numpy().astype("uint8"))

#for j in range(len(NombresEntrenoOt)):
#    
#    imagen=image.load_img(carpeta+os.sep+'otro'+os.sep+'otro'+os.sep+NombresEntrenoCo[j], grayscale=False, color_mode='rgb', target_size=(dimensiones[0],dimensiones[1]))
#    imagen=image.img_to_array(imagen)
#    imagen=np.array(imagen)
#    imagen.next()
#    #image = np.expand_dims(ndimage.imread(carpeta+os.sep+'otro'+os.sep+'otro'+os.sep+NombresEntrenoCo[j]), 0)
#    df.fit(imagen)
#    for x, val in zip(df.flow(imagen,save_to_dir=os.getcwd()+os.sep+"nuevas"+os.sep+"covid",save_prefix='Nueva',save_format='jpeg'),range(5)):
#        pass
    #df.flow(imagen,batch_size=3, save_to_dir=carpeta+os.sep+'otro', save_format='jpg',shuffle=True)


#df.fit(image)

pd= layers.ZeroPadding2D(padding=2)

Found 87 images belonging to 1 classes.
Found 70 images belonging to 1 classes.


In [8]:
print(len(cov))
print(len(otr))



18
14


In [12]:
for j in range(3):
    for i in range(len(cov)):
        cov.next()
    for i in range(len(otr)):
        otr.next()

##### Guardamos las imagenes en memoria

In [9]:

#entreno, val, test=image_dataset_from_directory(directory="images", image_size=(dimensiones[0],dimensiones[1]), subset="training", validation_split=0.2, seed=1234,
#                                      labels='inferred')



#entrenamiento
img_fit= image_dataset_from_directory(directory=nuevo, image_size=(dimensiones[0],dimensiones[1]), subset="training", validation_split=0.1, seed=1234,
                                      labels='inferred')


clases=img_fit.class_names

#validadcion
img_val=image_dataset_from_directory(directory=nuevo, image_size=(dimensiones[0],dimensiones[1]),subset="validation",validation_split=0.1, seed=1234,
                                      labels='inferred')
#clases=img_val.class_names

#Prediccion
img=image_dataset_from_directory(directory="perdidos", image_size=(dimensiones[0],dimensiones[1]), 
                                      labels='inferred')

#PARA MEJORAR EL RENDIMIENTO
AUTOTUNE = tf.data.AUTOTUNE
img_fit=img_fit.cache().prefetch(buffer_size=AUTOTUNE)
img_val=img_val.cache().prefetch(buffer_size=AUTOTUNE)

Normalizar= layers.experimental.preprocessing.Rescaling(1./255, input_shape=(dimensiones[0],dimensiones[1], 3))


Found 717 files belonging to 2 classes.
Using 646 files for training.
Found 717 files belonging to 2 classes.
Using 71 files for validation.
Found 8 files belonging to 2 classes.


### CREACION DEL MODELO

In [17]:
pool=3  #CON UN POOL MÁS GRANDE SE REDUCEN LOS PESOS
model= Sequential([
    aumento,
    Normalizar,
    pd,
    layers.Conv2D(filters=16,kernel_size=3, strides=1,activation='relu'),
    layers.MaxPooling2D(pool_size=pool),
    layers.Conv2D(filters=32,kernel_size=3, strides=1,activation='relu'),
    layers.MaxPooling2D(pool_size=pool),
    #pd,
    layers.Conv2D(filters=64,kernel_size=3, strides=1,activation='relu'),
    layers.MaxPooling2D(pool_size=pool),
    #pd,
    layers.Conv2D(filters=128,kernel_size=3, strides=1,activation='relu'),
    layers.MaxPooling2D(pool_size=pool),
    #pd,
    layers.Conv2D(filters=256,kernel_size=3, strides=1,activation='relu'),
    layers.MaxPooling2D(pool_size=pool),
    #pd,
    layers.Conv2D(filters=512,kernel_size=3, strides=1,activation='relu'),
    layers.MaxPooling2D(pool_size=pool),
    layers.Dropout(0.3),
    layers.Flatten(),
    Dense(500,'relu'),
    Dense(1,'sigmoid')

])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 1920, 1500, 3)     0         
_________________________________________________________________
rescaling (Rescaling)        (None, 1920, 1500, 3)     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 multiple                  0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 1922, 1502, 16)    448       
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 640, 500, 16)      0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 638, 498, 32)      4640      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 212, 166, 32)     

### ENTRENAMIENTO

In [16]:
opti= optimizers.Adam(learning_rate=0.11)
vueltas=10

model.compile(optimizer=opti, loss='categorical_crossentropy', metrics=['accuracy'])
historico=model.fit(img_fit, validation_data= img_val, epochs= vueltas)

Epoch 1/10
 1/21 [>.............................] - ETA: 36:36 - loss: 0.0000e+00 - accuracy: 0.6562

KeyboardInterrupt: 

##### RESULTADOS DEL ENTRENAMIENTO SIN DATA AUGMENTATION

In [ ]:


acc = historico.history['accuracy']
val_acc = historico.history['val_accuracy']

loss = historico.history['loss']
val_loss = historico.history['val_loss']

epochs_range = range(vueltas)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [23]:
#Con los datos aumentados

#historicoNu=model.fit(resul, validation_data= img_val, epochs=vueltas)

Epoch 1/20
53/53 [==============================] - 66s 1s/step - loss: nan - accuracy: 0.5541 - val_loss: nan - val_accuracy: 0.5806
Epoch 2/20
53/53 [==============================] - 69s 1s/step - loss: nan - accuracy: 0.5541 - val_loss: nan - val_accuracy: 0.5806
Epoch 3/20
53/53 [==============================] - 70s 1s/step - loss: nan - accuracy: 0.5541 - val_loss: nan - val_accuracy: 0.5806
Epoch 4/20
53/53 [==============================] - 69s 1s/step - loss: nan - accuracy: 0.5541 - val_loss: nan - val_accuracy: 0.5806
Epoch 5/20
53/53 [==============================] - 69s 1s/step - loss: nan - accuracy: 0.5541 - val_loss: nan - val_accuracy: 0.5806
Epoch 6/20
53/53 [==============================] - 69s 1s/step - loss: nan - accuracy: 0.5541 - val_loss: nan - val_accuracy: 0.5806
Epoch 7/20
53/53 [==============================] - 69s 1s/step - loss: nan - accuracy: 0.5541 - val_loss: nan - val_accuracy: 0.5806
Epoch 8/20
53/53 [==============================] - 69s 1s/ste

##### RESULTADOS DEL ENTRENAMIENTO CON LAS NUEVAS IMAGENES

In [22]:
#rutas=[ os.getcwd()+os.sep+'perdidos'+os.sep+'covid'+os.sep ,os.getcwd()+os.sep+'perdidos'+os.sep+'otro'+os.sep]
#nombres=noEstanco+noEstanot
#num=1
#for i in range(len(rutas)):
#    for j in range(len(nombres)):
#        print("\n",nombres[j])
#        if os.path.exists(rutas[i]+nombres[j]): 
#            
#            image = tf.keras.preprocessing.image.load_img(rutas[i]+nombres[j], target_size=(dimensiones[0],dimensiones[1]))
#            input_arr = keras.preprocessing.image.img_to_array(image)
#            input_arr = np.array([input_arr]) 
#            prediccion = model.predict(input_arr)
#            
#            score = tf.nn.sigmoid(prediccion[0])
#            print(
#                "La radiografia numero ",num, " pertenece a la clase {} con {:.2f} de precision."
#                .format(clases[np.argmax(score)], 100 * np.max(score))
#            )
#        num+=1

#Prediccion
#img=image_dataset_from_directory(directory="perdidos", image_size=(dimensiones[0],dimensiones[1]), 
                                      labels='inferred')
model.predict(img)



Found 8 files belonging to 2 classes.


array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]], dtype=float32)

In [35]:
test_loss, test_acc = model.evaluate(img,verbose=2)


1/1 - 0s - loss: nan - accuracy: 0.2500


TypeError: 'History' object is not subscriptable